In [1]:
import pandas as pd
import pymysql
import openpyxl
from openpyxl.styles import PatternFill, Border, Side, Font, Alignment
from datetime import datetime

In [4]:
def send_report_mail(to_list, cc_list, content_text, attach=None, title=None):
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.utils import formataddr
    import logging
    import datetime

    from email.mime.image import MIMEImage

    from email.header import Header
    from email.mime.base import MIMEBase
    from email import encoders

    def addimg(src, imgid):
        fp = open(src, 'rb')
        msgImage = MIMEImage(fp.read())
        fp.close()
        msgImage.add_header('Content-ID', imgid)
        return msgImage

    mail_host = "smtp.exmail.qq.com"
    mail_user = "report@iplas.com.cn"
    mail_pass = ''

    if title is None:
        sub = datetime.datetime.strftime(datetime.datetime.now(), "%Y%m%d") + "无标题"
    else:
        sub = title

    content_html = f"""<html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>{sub}</title>
        </head>
        <body>
            <div class="content">
            """

    content_html += f"""
                <p>{content_text}</p>
                """

    content_html += f'<p><br><img src="cid:image"></p>'

    content_html += """
            </div>
        </body>
        <style>
            .content{{padding: 0 20px;}}
            h3{{font-size: 17px; margin: 30px 0;}}
            h3, h4{{font-weight: normal;}}
            .red-text{{color: red;}}
            .bold-font{{font-weight: bold;}}
            .line-text p{{margin: 7px 0;}}
        </style>
    </html>
    """

    msg = MIMEMultipart()
    msg['From'] = formataddr(pair=("互联网管理中心智能报表平台", mail_user))
    msg['To'] = ';'.join(to_list)
    msg['Cc'] = ';'.join(cc_list)
    msg['Subject'] = sub
    msg.attach(MIMEText(content_html, 'html', 'utf-8'))
    msg["Accept-Language"] = "zh-CN"
    msg["Accept-Charset"] = "ISO-8859-1,utf-8"
    msg.attach(addimg(f'HDPE中空料.png', "image"))

    if attach is not None:
        with open(attach, 'rb') as f:
            appendix = MIMEBase('application', 'octet-stream')
            appendix.set_payload(f.read())
            encoders.encode_base64(appendix)
            filename = attach.split('/')[-1]
            appendix.add_header('Content-Disposition', 'attachment',
                                filename=('gbk', '', filename))
            msg.attach(appendix)

    try:
        s = smtplib.SMTP()
        s.connect(mail_host)
        s.login(mail_user, mail_pass)
        s.sendmail(mail_user, to_list + cc_list, msg.as_string())
        s.close()
        logging.info(f"收件人：{to_list}，抄送人：{cc_list}，邮件主题：{sub}，邮件正文：{content_text},已发送")
    except Exception as e:
        logging.error(e)



In [5]:

send_report_mail(["duanyang@iplas.com.cn"],["duanyang@iplas.com.cn"], content_text=None,title=None,attach=None)